In [ ]:
!pip install pennylane

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 957 kB 30.0 MB/s 
     |████████████████████████████████| 1.6 MB 22.6 MB/s 
     |████████████████████████████████| 9.3 MB 44.5 MB/s 
     |████████████████████████████████| 108 kB 62.9 MB/s 


In [31]:
import pandas as pd
import numpy as np
from pennylane import numpy as np
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler

import pennylane as qml
from pennylane.templates.embeddings import AngleEmbedding, AmplitudeEmbedding
from pennylane.optimize import AdamOptimizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


import time
start = time.time()

In [32]:
df = pd.read_csv('/content/ClaMP_Integrated-5184.csv', sep=',')

In [33]:
df.drop(['packer_type'],axis=1,inplace=True)

In [34]:
from sklearn.utils import shuffle
df = shuffle(df)

In [35]:
# Separate class
class_0 = df[df['class'] == 0]
class_1 = df[df['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (2488, 69)
class 1: (2722, 69)


In [36]:
df = df.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')

In [37]:
df.drop(['index'],axis=1,inplace=True)

In [38]:
df

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,80,2,4,65535,184,256,8,1,0,1,...,1,4,4,1,0.000000,0.000000,458240,6.276033,0,1
1,144,3,4,65535,184,240,4,1,1,1,...,1,0,4,0,7.484349,7.748603,140800,7.328307,1,1
2,144,3,4,65535,184,240,4,1,0,1,...,1,1,3,0,6.642822,5.566747,2388176,6.688557,1,0
3,144,3,4,65535,184,240,4,1,1,1,...,1,0,4,0,7.764590,7.882488,130048,7.474767,1,1
4,144,3,4,65535,184,208,4,1,1,1,...,1,0,4,0,7.967267,2.975948,352256,7.773508,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5205,144,3,4,65535,184,224,3,1,1,1,...,1,0,3,0,6.509278,0.487867,16883056,7.999597,1,0
5206,144,3,4,65535,184,232,6,1,0,1,...,1,2,4,0,6.403159,0.000000,93696,7.637015,1,1
5207,144,3,4,65535,184,128,4,1,0,1,...,1,1,3,0,7.939881,7.897105,161280,7.905354,0,1
5208,144,3,4,65535,184,248,4,1,0,1,...,1,1,3,0,6.437962,3.954410,76624,6.653188,1,0


In [39]:
df_5percent=df[0:3907]

In [40]:
df_5percent

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,80,2,4,65535,184,256,8,1,0,1,...,1,4,4,1,0.000000,0.000000,458240,6.276033,0,1
1,144,3,4,65535,184,240,4,1,1,1,...,1,0,4,0,7.484349,7.748603,140800,7.328307,1,1
2,144,3,4,65535,184,240,4,1,0,1,...,1,1,3,0,6.642822,5.566747,2388176,6.688557,1,0
3,144,3,4,65535,184,240,4,1,1,1,...,1,0,4,0,7.764590,7.882488,130048,7.474767,1,1
4,144,3,4,65535,184,208,4,1,1,1,...,1,0,4,0,7.967267,2.975948,352256,7.773508,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3902,144,3,4,65535,184,216,4,1,1,1,...,1,0,4,0,6.535118,1.581645,454656,7.414210,0,1
3903,144,3,4,65535,184,128,7,1,1,1,...,1,2,5,0,5.289852,2.490701,16896,6.119038,0,1
3904,144,3,4,65535,184,128,3,1,0,1,...,1,1,2,1,5.675380,0.000000,32768,4.116478,0,0
3905,144,3,4,65535,184,224,5,1,0,1,...,1,1,4,0,6.511254,3.978841,172032,7.283821,0,1


In [41]:
# Separate class
class_0 = df_5percent[df_5percent['class'] == 0]
class_1 = df_5percent[df_5percent['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (1865, 69)
class 1: (2042, 69)


In [42]:
# Read out CSV and sets/samples creation

df_5percent = df_5percent.astype(float)
train,test = train_test_split(df_5percent, test_size=0.30, random_state=2)
train_set = train
test_set = test
train_set = train_set.sample(160)
test_set = test_set.sample(40)
np.random.seed(42)

In [43]:
df_5percent.info

<bound method DataFrame.info of       e_cblp  e_cp  e_cparhdr  e_maxalloc   e_sp  e_lfanew  NumberOfSections  \
0       80.0   2.0        4.0     65535.0  184.0     256.0               8.0   
1      144.0   3.0        4.0     65535.0  184.0     240.0               4.0   
2      144.0   3.0        4.0     65535.0  184.0     240.0               4.0   
3      144.0   3.0        4.0     65535.0  184.0     240.0               4.0   
4      144.0   3.0        4.0     65535.0  184.0     208.0               4.0   
...      ...   ...        ...         ...    ...       ...               ...   
3902   144.0   3.0        4.0     65535.0  184.0     216.0               4.0   
3903   144.0   3.0        4.0     65535.0  184.0     128.0               7.0   
3904   144.0   3.0        4.0     65535.0  184.0     128.0               3.0   
3905   144.0   3.0        4.0     65535.0  184.0     224.0               5.0   
3906   144.0   3.0        4.0     65535.0  184.0     256.0               4.0   

      C

In [44]:
df_5percent.describe()

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
count,3907.000000,3907.000000,3907.000000,3907.000000,3907.000000,3907.000000,3907.000000,3907.000000,3907.000000,3907.0,...,3907.000000,3907.000000,3907.000000,3907.000000,3907.000000,3907.000000,3.907000e+03,3907.000000,3907.000000,3907.000000
mean,152.732275,8.076529,7.257998,65141.971078,204.280010,223.064244,4.650115,0.990530,0.367289,1.0,...,0.998976,1.353468,3.296647,0.153826,4.934696,2.534879,7.916796e+05,6.364991,0.544919,0.522652
std,653.831090,320.730232,205.533042,5016.183017,1082.686042,48.539475,1.911556,0.096865,0.482128,0.0,...,0.031985,1.621102,1.134027,0.360829,2.523866,2.694553,5.664449e+06,1.120003,0.498042,0.499551
min,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000,1.000000,0.000000,0.000000,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.536000e+03,0.939626,0.000000,0.000000
25%,144.000000,3.000000,4.000000,65535.000000,184.000000,208.000000,4.000000,1.000000,0.000000,1.0,...,1.000000,1.000000,3.000000,0.000000,3.886006,0.000000,6.092800e+04,5.690993,0.000000,0.000000
50%,144.000000,3.000000,4.000000,65535.000000,184.000000,232.000000,5.000000,1.000000,0.000000,1.0,...,1.000000,1.000000,4.000000,0.000000,6.153046,1.521239,1.228800e+05,6.393665,1.000000,1.000000
75%,144.000000,3.000000,4.000000,65535.000000,184.000000,248.000000,5.000000,1.000000,1.000000,1.0,...,1.000000,2.000000,4.000000,0.000000,6.509278,4.702634,3.112960e+05,7.304325,1.000000,1.000000
max,37008.000000,20050.000000,12851.000000,65535.000000,65534.000000,648.000000,34.000000,1.000000,1.000000,1.0,...,1.000000,31.000000,7.000000,1.000000,7.999859,7.999620,1.657081e+08,7.999997,1.000000,1.000000


In [45]:
# Separation of labels

x_train = train_set
y_train = train_set[['class']]

x_test = test_set
y_test = test_set[['class']]

In [47]:
# Reduce dimensions using PCA to fit the dimensions with the qubits

n_dim = 2
pca = PCA(n_components=n_dim)
pca.fit(x_train)

x_train = pca.transform(x_train)

pca.fit(x_test)
x_test = pca.transform(x_test)

In [48]:
# Normalize

std_scale = StandardScaler().fit(x_train)
data = std_scale.transform(x_train)

std_scale = StandardScaler().fit(x_test)
x_test = std_scale.transform(x_test)

In [49]:
# Review the balance of the target variable in train

y_train.value_counts(normalize=True)*100

class
1.0      53.75
0.0      46.25
dtype: float64

In [50]:
# Review the balance of the target variable in test

y_test.value_counts(normalize=True)*100

class
1.0      55.0
0.0      45.0
dtype: float64

In [51]:
# Angle Encoding

num_qubits = n_dim

dev = qml.device('default.qubit', wires = num_qubits)

@qml.qnode(dev)
def circuit(parameters, data):
    for i in range(num_qubits):
        qml.Hadamard(wires = i)
    
    AngleEmbedding(features = data, wires = range(num_qubits), rotation = 'Y')
    
    qml.StronglyEntanglingLayers(weights = parameters, wires = range(num_qubits))
    
    return qml.expval(qml.PauliZ(0))

In [52]:
num_layers = 5
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

print(weights_init, bias_init)

[[[ 0.00496714 -0.00138264  0.00647689]
  [ 0.0152303  -0.00234153 -0.00234137]]

 [[ 0.01579213  0.00767435 -0.00469474]
  [ 0.0054256  -0.00463418 -0.0046573 ]]

 [[ 0.00241962 -0.0191328  -0.01724918]
  [-0.00562288 -0.01012831  0.00314247]]

 [[-0.00908024 -0.01412304  0.01465649]
  [-0.00225776  0.00067528 -0.01424748]]

 [[-0.00544383  0.00110923 -0.01150994]
  [ 0.00375698 -0.00600639 -0.00291694]]] 0.0


In [53]:
circuit(weights_init, data[0])

tensor(0.07937633, requires_grad=True)

In [54]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

In [55]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

In [56]:
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [57]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

In [58]:
Y = np.array(y_train.values[:,0] * 2 - np.ones(len(y_train.values[:,0])), requires_grad = False)  # shift label from {0, 1} to {-1, 1}
X = np.array(data, requires_grad=False)

for i in range(5):
    print("X = {}, Y = {: d}".format(list(X[i]), int(Y[i])))

X = [tensor(-0.1078656, requires_grad=False), tensor(-0.43027909, requires_grad=False)], Y = -1
X = [tensor(-0.06780275, requires_grad=False), tensor(-0.29014525, requires_grad=False)], Y = -1
X = [tensor(0.40032125, requires_grad=False), tensor(-0.40296422, requires_grad=False)], Y = -1
X = [tensor(-0.12541356, requires_grad=False), tensor(-0.29251103, requires_grad=False)], Y =  1
X = [tensor(-0.178177, requires_grad=False), tensor(3.3977491, requires_grad=False)], Y =  1


In [59]:
opt = AdamOptimizer(stepsize=0.1, beta1=0.9, beta2=0.99, eps=1e-08)
batch_size = 10

In [60]:
%%time
weights = weights_init
bias = bias_init

wbest = 0
bbest = 0
abest = 0

for it in range(100):

    # weights update by one optimizer step

    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # Compute the accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]
    
    if accuracy(Y, predictions) > abest:
        wbest = weights
        bbest = bias
        abest = accuracy(Y, predictions)
        print('New best')

    acc = accuracy(Y, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X, Y), acc
        )
    )

New best
Iter:     1 | Cost: 1.1027027 | Accuracy: 0.4625000 
Iter:     2 | Cost: 1.0860744 | Accuracy: 0.4625000 
New best
Iter:     3 | Cost: 0.9638965 | Accuracy: 0.5750000 
Iter:     4 | Cost: 0.9967222 | Accuracy: 0.5625000 
Iter:     5 | Cost: 1.0258449 | Accuracy: 0.5562500 
Iter:     6 | Cost: 0.9638472 | Accuracy: 0.5625000 
New best
Iter:     7 | Cost: 0.9265283 | Accuracy: 0.5875000 
New best
Iter:     8 | Cost: 0.9163396 | Accuracy: 0.5937500 
Iter:     9 | Cost: 0.9186371 | Accuracy: 0.5437500 
Iter:    10 | Cost: 0.9653419 | Accuracy: 0.5375000 
Iter:    11 | Cost: 1.0383259 | Accuracy: 0.5375000 
Iter:    12 | Cost: 1.0065824 | Accuracy: 0.5375000 
Iter:    13 | Cost: 0.9355817 | Accuracy: 0.5437500 
Iter:    14 | Cost: 0.9252339 | Accuracy: 0.5875000 
Iter:    15 | Cost: 1.0808159 | Accuracy: 0.5562500 
Iter:    16 | Cost: 1.3325774 | Accuracy: 0.5500000 
Iter:    17 | Cost: 1.4073828 | Accuracy: 0.5437500 
Iter:    18 | Cost: 1.3196280 | Accuracy: 0.5437500 
Iter:    1

In [61]:
Yte = np.array(y_test.values[:,0] * 2 - np.ones(len(y_test.values[:,0])), requires_grad = False)
Xte = np.array(normalize(x_test), requires_grad=False)

In [62]:
predictions = [np.sign(variational_classifier(wbest, bbest, x)) for x in Xte]
pred = [np.sign(variational_classifier(wbest, bbest, x)) for x in X]
acc = accuracy(Yte, predictions)

print(f'Cost: {cost(wbest, bbest, Xte, Yte)}, Accuracy: {np.round(acc, 2) * 100}%')

Cost: 0.772033226553561, Accuracy: 70.0%


In [63]:
pd.DataFrame((predictions, Yte), ('Predictions', 'Test')).T

,Predictions,Test
0,-1.0,-1.0
1,1.0,-1.0
2,-1.0,-1.0
3,1.0,1.0
4,1.0,1.0
5,1.0,-1.0
6,1.0,-1.0
7,1.0,-1.0
8,1.0,1.0
9,1.0,1.0


In [64]:
# Print the classification report and important metrics

print(metrics.classification_report(predictions,Yte))
print(metrics.precision_score(predictions,Yte))
print(metrics.recall_score(predictions,Yte))
print(metrics.f1_score(predictions,Yte))
print(metrics.balanced_accuracy_score(predictions,Yte))

              precision    recall  f1-score   support

        -1.0       0.33      1.00      0.50         6
         1.0       1.00      0.65      0.79        34

    accuracy                           0.70        40
   macro avg       0.67      0.82      0.64        40
weighted avg       0.90      0.70      0.74        40

1.0
0.6470588235294118
0.7857142857142858
0.8235294117647058
